# Tesla Top Posts Scraper

Scrapt die 5 meist gevotetsten Tesla-Posts aus verschiedenen Subreddits.

In [39]:
import praw
import pandas as pd
from datetime import datetime
import re
import time
import os
from dotenv import load_dotenv

In [40]:
# Reddit API Setup - Verwendung der .env-Datei (wie in reddit_scraper.ipynb)
from dotenv import load_dotenv
import os
import praw

# Lade Umgebungsvariablen aus .env-Datei
load_dotenv()

# Reddit API Credentials aus .env-Datei
reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent=os.getenv("REDDIT_USER_AGENT")
)

# SETTINGS
keywords = [
    'tesla',
    'tsla', 
    'elon',
    'musk'
]

# Subreddits zum Durchsuchen
subreddits = [
    'wallstreetbets',
    'stocks', 
    'investing',
    'SecurityAnalysis',
    'StockMarket',
    'ValueInvesting',
    'technology',
    'electricvehicles',
    'teslamotors',
    'elonmusk',
    'GME',
    'Aktien',
    'teslainvestorsclub',
    'wallstreetbetsGER'
]

# Anzahl Top Posts die am Ende zurückgegeben werden sollen
TOP_POSTS_COUNT = 5

In [41]:
def contains_tesla_keywords(text, keywords):
    """
    Prüft ob Text Tesla-Keywords enthält
    """
    if pd.isna(text) or text is None:
        return False
        
    # Regex Pattern erstellen (Wortgrenzen für genauere Matches)
    pattern = r'\b(' + '|'.join(keywords) + r')\b'
    return bool(re.search(pattern, str(text), re.IGNORECASE))

def get_top_tesla_posts(subreddits, keywords, top_count=5):
    """
    Holt die Top Tesla-Posts aus allen Subreddits und gibt die meist gevotetsten zurück
    """
    all_posts = []
    
    # Durchsuche alle Subreddits
    for subreddit_name in subreddits:
        try:
            subreddit = reddit.subreddit(subreddit_name)
            for post in subreddit.hot(limit=50):
                # Prüfe Titel und Text auf Tesla Keywords
                if (contains_tesla_keywords(post.title, keywords) or 
                    contains_tesla_keywords(post.selftext, keywords)):
                    
                    title = post.title
                    text = post.selftext if post.selftext else ''
                    full_text = f"{title} {text}".strip()
                    
                    all_posts.append({
                        'title': title,
                        'text': text,
                        'score': post.score,
                        'created': datetime.fromtimestamp(post.created_utc),
                        'full_text': full_text
                    })
            # Kurze Pause zwischen Requests
            time.sleep(1)
        except Exception as e:
            print(f"Fehler beim Scrapen von r/{subreddit_name}: {e}")
    
    # Sortiere nach Score (Upvotes) 
    all_posts.sort(key=lambda x: x['score'], reverse=True)
    
    # Nehme die Top N Posts
    return pd.DataFrame(all_posts[:top_count])

In [42]:
tesla_df = get_top_tesla_posts(subreddits, keywords, TOP_POSTS_COUNT)

if not tesla_df.empty:
    print(f"Found {len(tesla_df)} Tesla posts")
    for idx, row in tesla_df.iterrows():
        print(f"{idx+1}. [{row['score']} upvotes] {row['title'][:60]}...")
else:
    print("No Tesla posts found")

Found 5 Tesla posts
1. [23889 upvotes] Tesla is the least-trusted car brand in America, survey find...
2. [3396 upvotes] Grok is officially coming to Tesla next week at the latest....
3. [2328 upvotes] Tesla Cybertruck Sales Fall Dramatically To Their Lowest Lev...
4. [2095 upvotes] Elon pinned x: "It is obvious with the insane spending of th...
5. [1711 upvotes] Optimus handing out popcorn at the Tesla Diner...


In [43]:
# Export data
if not tesla_df.empty:
    os.makedirs('data', exist_ok=True)
    filename = "data/tesla_top5_this_week.csv"
    tesla_df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")
else:
    print("No data to export")

Data saved to data/tesla_top5_this_week.csv


In [44]:
# Aktueller Tesla Aktienstand
try:
    import requests
    
    url = 'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol=TSLA&apikey=GR4S16Y4XV97PF9Z'
    response = requests.get(url)
    stock_data = response.json()
    
    if 'Global Quote' in stock_data:
        quote = stock_data['Global Quote']
        current_price = float(quote['05. price'])
        change = float(quote['09. change'])
        change_percent = quote['10. change percent'].rstrip('%')
        
        print(f"Tesla Stock: ${current_price:.2f} ({change:+.2f}, {change_percent}%)")
        
        if not tesla_df.empty:
            tesla_df['tesla_stock_price'] = current_price
            tesla_df['tesla_stock_change'] = change
    else:
        print("Error fetching Tesla stock price")
        
except Exception as e:
    print(f"Stock API error: {e}")

Tesla Stock: $332.11 (+3.62, 1.1020%)
